# 1-S10: Transformer for Tabular Data

정형(Tabular) 데이터에 Transformer를 적용하는 방법을 학습합니다.

## 학습 목표
1. **Self-Attention** - Query, Key, Value 개념
2. **정형 데이터에서 Transformer** - NLP와의 차이점
3. **TabTransformer** - 범주형 피처에 Transformer 적용
4. **FT-Transformer** - 모든 피처에 Transformer 적용
5. **트리 vs Transformer** - 언제 Transformer가 유리한가?

## 왜 Transformer인가?

| 모델 | 장점 | 단점 |
|------|------|------|
| 트리 (XGBoost) | 빠름, 해석 가능 | 피처 간 상호작용 제한 |
| **Transformer** | 피처 간 관계 학습 | 느림, 데이터 많이 필요 |

## 예상 시간
약 30분

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("패키지 로드 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"GPU 사용 가능: {torch.cuda.is_available()}")

---
## 1. Self-Attention 기초

### 1-1. Attention이란?

**핵심 아이디어:** "어떤 정보에 집중할지 학습"

```
입력: [거래금액, 거래시간, 카드종류, 위치]
      ↓
Attention: "이 거래에서 '위치'와 '카드종류'가 중요하다"
      ↓
가중치: [0.1, 0.2, 0.3, 0.4]  # 위치에 높은 가중치
```

### 1-2. Query, Key, Value (Q, K, V)

**비유: 검색 엔진**

| 요소 | 역할 | 비유 |
|------|------|------|
| **Query (Q)** | "무엇을 찾고 있는가?" | 검색어 |
| **Key (K)** | "무엇이 있는가?" | 문서 제목 |
| **Value (V)** | "실제 정보" | 문서 내용 |

### 1-3. Self-Attention 수식

```
Attention(Q, K, V) = softmax(Q·K^T / √d_k) · V
```

1. Q와 K의 내적 → 유사도 계산
2. √d_k로 나눔 → 스케일 조정
3. softmax → 확률로 변환 (합이 1)
4. V와 곱함 → 가중합

In [ ]:
# Self-Attention 직접 구현

def scaled_dot_product_attention(Q, K, V):
    """
    Scaled Dot-Product Attention
    
    Args:
        Q: Query (batch, seq_len, d_k)
        K: Key (batch, seq_len, d_k)
        V: Value (batch, seq_len, d_v)
    
    Returns:
        output: Attention output
        attention_weights: 어디에 집중했는지
    """
    d_k = Q.size(-1)
    
    # 1. Q·K^T (유사도 계산)
    scores = torch.matmul(Q, K.transpose(-2, -1))
    
    # 2. Scale (기울기 안정화)
    scores = scores / np.sqrt(d_k)
    
    # 3. Softmax (확률로 변환)
    attention_weights = F.softmax(scores, dim=-1)
    
    # 4. V와 곱함 (가중합)
    output = torch.matmul(attention_weights, V)
    
    return output, attention_weights

print("Self-Attention 함수 정의 완료!")

In [ ]:
# Self-Attention 예제 (4개 피처)

# 4개 피처: [거래금액, 거래시간, 카드종류, 위치]
# 각 피처를 4차원 임베딩으로 표현
seq_len = 4  # 피처 수
d_model = 4  # 임베딩 차원

# 임의의 입력 (1개 샘플)
torch.manual_seed(42)
X = torch.randn(1, seq_len, d_model)

# Q, K, V 생성 (실제로는 학습되는 가중치로 변환)
# 여기서는 간단히 X를 그대로 사용
Q, K, V = X, X, X

# Self-Attention 적용
output, attention_weights = scaled_dot_product_attention(Q, K, V)

print("입력 shape:", X.shape)
print("출력 shape:", output.shape)
print("\nAttention Weights (어디에 집중했는지):")
print(attention_weights[0].numpy().round(3))

In [ ]:
# Attention Weights 시각화

feature_names = ['거래금액', '거래시간', '카드종류', '위치']

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(attention_weights[0].detach().numpy(), cmap='Blues')

ax.set_xticks(range(len(feature_names)))
ax.set_yticks(range(len(feature_names)))
ax.set_xticklabels(feature_names)
ax.set_yticklabels(feature_names)

ax.set_xlabel('Key (정보 제공)')
ax.set_ylabel('Query (정보 요청)')
ax.set_title('Self-Attention Weights\n(각 피처가 다른 피처에 얼마나 집중하는가)')

# 값 표시
for i in range(len(feature_names)):
    for j in range(len(feature_names)):
        val = attention_weights[0, i, j].item()
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', 
                color='white' if val > 0.3 else 'black')

plt.colorbar(im)
plt.tight_layout()
plt.show()

print("\n해석: 각 행은 Query 피처가 Key 피처들에 얼마나 집중하는지 보여줌")
print("      대각선이 높으면 자기 자신에 집중 (Self-Attention)")

### 1-4. Multi-Head Attention

**문제:** 단일 Attention은 하나의 관점만 학습

**해결:** 여러 개의 Attention을 병렬로 (Multi-Head)

```
Head 1: "거래금액과 위치의 관계" 학습
Head 2: "거래시간과 카드종류의 관계" 학습
Head 3: "전체적인 패턴" 학습
...
→ 모든 Head 결과를 concat → 풍부한 표현
```

```python
# PyTorch에서
multihead_attn = nn.MultiheadAttention(embed_dim=64, num_heads=8)
```

---
## 2. 정형 데이터에서 Transformer

### 2-1. NLP vs 정형 데이터

| 특성 | NLP (텍스트) | Tabular (정형) |
|------|-------------|----------------|
| 입력 | 단어 시퀀스 | 피처 집합 |
| **순서** | 중요 (문장 순서) | **순서 없음** |
| 토큰 | 단어/서브워드 | 피처 값 |
| Positional Encoding | 필수 | **불필요** |

### 2-2. Column Embedding

**아이디어:** 각 피처(컬럼)를 하나의 "토큰"으로 취급

```
NLP:     ["I", "love", "AI"]  → [embed("I"), embed("love"), embed("AI")]
Tabular: [100, 0.5, "visa"]   → [embed(금액), embed(시간), embed(카드)]
```

### 2-3. 수치형 vs 범주형 처리

| 타입 | 처리 방법 |
|------|----------|
| **수치형** | Linear projection → 임베딩 |
| **범주형** | Embedding lookup (NLP와 동일) |

---
## 3. TabTransformer (2020)

**논문:** "TabTransformer: Tabular Data Modeling Using Contextual Embeddings"

### 3-1. 핵심 아이디어

**범주형 피처만** Transformer로 처리

```
입력: [수치형 피처들] + [범주형 피처들]
              ↓                ↓
           그대로         Transformer
              ↓                ↓
           Concat ← ← ← ← ← ←
              ↓
            MLP
              ↓
            예측
```

### 3-2. 왜 범주형만?

- 범주형 피처 간 상호작용이 중요
- 예: "카드종류=visa" + "국가=해외" → 사기 확률 상승
- Transformer가 이런 조합을 학습

### 3-3. 구조

```
1. Column Embedding: 각 범주형 피처를 d차원 벡터로
2. Transformer Layers: 피처 간 관계 학습 (N개 레이어)
3. Concatenation: Transformer 출력 + 수치형 피처
4. MLP: 최종 분류/회귀
```

In [ ]:
# TabTransformer 간단 구현

class TabTransformer(nn.Module):
    """
    TabTransformer: 범주형 피처에 Transformer 적용
    
    Args:
        num_categories: 각 범주형 피처의 카테고리 수 리스트
        num_continuous: 수치형 피처 수
        d_model: 임베딩 차원
        n_heads: Multi-Head Attention 헤드 수
        n_layers: Transformer 레이어 수
    """
    
    def __init__(self, num_categories, num_continuous, d_model=32, n_heads=4, n_layers=2):
        super().__init__()
        
        self.num_categories = num_categories
        self.num_continuous = num_continuous
        self.d_model = d_model
        
        # 범주형 임베딩 (각 피처별로)
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_cat, d_model) for num_cat in num_categories
        ])
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # MLP (Transformer 출력 + 수치형 피처 → 예측)
        mlp_input_dim = len(num_categories) * d_model + num_continuous
        self.mlp = nn.Sequential(
            nn.Linear(mlp_input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x_cat, x_cont):
        """
        Args:
            x_cat: 범주형 피처 (batch, num_cat_features) - 정수 인덱스
            x_cont: 수치형 피처 (batch, num_cont_features) - 실수
        """
        # 1. 범주형 임베딩
        cat_embeds = []
        for i, embed in enumerate(self.embeddings):
            cat_embeds.append(embed(x_cat[:, i]))
        
        # (batch, num_cat_features, d_model)
        cat_embeds = torch.stack(cat_embeds, dim=1)
        
        # 2. Transformer (범주형 피처 간 관계 학습)
        transformed = self.transformer(cat_embeds)
        
        # 3. Flatten
        transformed = transformed.flatten(start_dim=1)
        
        # 4. Concat (Transformer 출력 + 수치형)
        combined = torch.cat([transformed, x_cont], dim=1)
        
        # 5. MLP
        out = self.mlp(combined)
        
        return out.squeeze(-1)

print("TabTransformer 클래스 정의 완료!")

In [ ]:
# TabTransformer 테스트

# 예시: 3개 범주형 피처, 5개 수치형 피처
num_categories = [10, 5, 8]  # 각 범주형 피처의 카테고리 수
num_continuous = 5

model = TabTransformer(
    num_categories=num_categories,
    num_continuous=num_continuous,
    d_model=32,
    n_heads=4,
    n_layers=2
)

# 더미 데이터
batch_size = 4
x_cat = torch.randint(0, 5, (batch_size, len(num_categories)))  # 범주형 인덱스
x_cont = torch.randn(batch_size, num_continuous)  # 수치형

# Forward
output = model(x_cat, x_cont)

print(f"범주형 입력: {x_cat.shape}")
print(f"수치형 입력: {x_cont.shape}")
print(f"출력: {output.shape}")
print(f"예측 확률: {output.detach().numpy().round(3)}")

# 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
print(f"\n총 파라미터 수: {total_params:,}")

---
## 4. FT-Transformer (2021)

**논문:** "Revisiting Deep Learning Models for Tabular Data"

### 4-1. 핵심 아이디어

**모든 피처**를 Transformer로 처리 (수치형 포함)

```
TabTransformer: 범주형만 Transformer
FT-Transformer: 모든 피처 Transformer
```

### 4-2. Feature Tokenizer

수치형 피처도 "토큰"으로 변환:

```
수치형 피처 x → Linear(x) + bias → d차원 임베딩
범주형 피처 c → Embedding(c) → d차원 임베딩
```

### 4-3. [CLS] 토큰

BERT처럼 특별한 [CLS] 토큰 추가:

```
입력: [[CLS], 피처1, 피처2, ..., 피처N]
       ↓
Transformer
       ↓
[CLS]의 출력 → 예측에 사용
```

### 4-4. 구조 비교

| 요소 | TabTransformer | FT-Transformer |
|------|----------------|----------------|
| 수치형 처리 | MLP에서 | **Transformer에서** |
| 범주형 처리 | Transformer | Transformer |
| 출력 방식 | Flatten | **[CLS] 토큰** |
| 성능 | 좋음 | **더 좋음** |

In [ ]:
# FT-Transformer 간단 구현

class FTTransformer(nn.Module):
    """
    FT-Transformer: 모든 피처에 Transformer 적용
    
    Args:
        num_categories: 각 범주형 피처의 카테고리 수 리스트
        num_continuous: 수치형 피처 수
        d_model: 임베딩 차원
        n_heads: Multi-Head Attention 헤드 수
        n_layers: Transformer 레이어 수
    """
    
    def __init__(self, num_categories, num_continuous, d_model=32, n_heads=4, n_layers=2):
        super().__init__()
        
        self.num_categories = num_categories
        self.num_continuous = num_continuous
        self.d_model = d_model
        
        # 범주형 임베딩
        self.cat_embeddings = nn.ModuleList([
            nn.Embedding(num_cat, d_model) for num_cat in num_categories
        ])
        
        # 수치형 Feature Tokenizer (각 피처별 Linear)
        self.cont_embeddings = nn.ModuleList([
            nn.Linear(1, d_model) for _ in range(num_continuous)
        ])
        
        # [CLS] 토큰
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # 출력 레이어 ([CLS] 토큰 → 예측)
        self.output = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x_cat, x_cont):
        """
        Args:
            x_cat: 범주형 피처 (batch, num_cat_features)
            x_cont: 수치형 피처 (batch, num_cont_features)
        """
        batch_size = x_cat.size(0)
        
        # 1. 범주형 임베딩
        cat_embeds = [embed(x_cat[:, i]) for i, embed in enumerate(self.cat_embeddings)]
        
        # 2. 수치형 임베딩 (Feature Tokenizer)
        cont_embeds = [
            embed(x_cont[:, i:i+1]) for i, embed in enumerate(self.cont_embeddings)
        ]
        
        # 3. 모든 임베딩 결합
        all_embeds = cat_embeds + cont_embeds
        tokens = torch.stack(all_embeds, dim=1)  # (batch, num_features, d_model)
        
        # 4. [CLS] 토큰 추가
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        tokens = torch.cat([cls_tokens, tokens], dim=1)  # (batch, 1+num_features, d_model)
        
        # 5. Transformer
        transformed = self.transformer(tokens)
        
        # 6. [CLS] 토큰 출력만 사용
        cls_output = transformed[:, 0]  # (batch, d_model)
        
        # 7. 예측
        out = self.output(cls_output)
        
        return out.squeeze(-1)

print("FT-Transformer 클래스 정의 완료!")

In [ ]:
# FT-Transformer 테스트

model_ft = FTTransformer(
    num_categories=num_categories,
    num_continuous=num_continuous,
    d_model=32,
    n_heads=4,
    n_layers=2
)

# Forward
output_ft = model_ft(x_cat, x_cont)

print(f"출력: {output_ft.shape}")
print(f"예측 확률: {output_ft.detach().numpy().round(3)}")

# 파라미터 수 비교
params_tab = sum(p.numel() for p in model.parameters())
params_ft = sum(p.numel() for p in model_ft.parameters())

print(f"\nTabTransformer 파라미터: {params_tab:,}")
print(f"FT-Transformer 파라미터: {params_ft:,}")
print(f"→ FT-Transformer가 수치형 임베딩으로 인해 더 많음")

---
## 5. 트리 모델 vs Transformer

### 5-1. 2024-2025 벤치마크 결과

**"Why do tree-based models still outperform deep learning on typical tabular data?"**

| 데이터 크기 | 승자 | 이유 |
|------------|------|------|
| **작은 데이터 (<10K)** | 트리 | DL은 과적합 |
| **중간 데이터 (10K-100K)** | 트리 ≈ DL | 비슷 |
| **큰 데이터 (>100K)** | DL 유리 | 충분한 학습 |

### 5-2. 언제 Transformer가 유리한가?

1. **대규모 데이터** (100K+ 샘플)
2. **피처 간 복잡한 상호작용**이 있을 때
3. **범주형 피처가 많을 때**
4. **Semi-supervised Learning** 가능할 때

### 5-3. 트리가 유리한 경우

1. **작은/중간 데이터**
2. **해석 가능성** 중요
3. **빠른 추론** 필요
4. **튜닝 시간** 제한

### 5-4. FDS에서의 선택

| 요소 | 분석 | 결론 |
|------|------|------|
| 데이터 크기 | 590K | Transformer 가능 |
| 추론 속도 | 실시간 필요 | 트리 유리 |
| 해석 가능성 | 규제 요구 | 트리 유리 (SHAP) |
| **최종** | - | **트리 스태킹 + Transformer 실험** |

In [ ]:
# 트리 vs Transformer 비교 시각화

comparison_data = {
    '모델': ['XGBoost', 'LightGBM', 'CatBoost', 'TabTransformer', 'FT-Transformer'],
    '학습 속도': [5, 5, 4, 2, 2],
    '추론 속도': [5, 5, 4, 3, 3],
    '해석 가능성': [5, 5, 5, 2, 2],
    '대규모 데이터': [4, 4, 4, 5, 5],
    '피처 상호작용': [3, 3, 3, 4, 5]
}

df_compare = pd.DataFrame(comparison_data)
df_compare = df_compare.set_index('모델')

# Radar Chart
categories = list(df_compare.columns)
N = len(categories)

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # 닫기

colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6', '#f39c12']

for idx, model in enumerate(df_compare.index):
    values = df_compare.loc[model].values.flatten().tolist()
    values += values[:1]  # 닫기
    ax.plot(angles, values, 'o-', linewidth=2, label=model, color=colors[idx])
    ax.fill(angles, values, alpha=0.1, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 5)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.set_title('트리 vs Transformer 비교', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n결론:")
print("  - 트리: 속도, 해석 가능성에서 우위")
print("  - Transformer: 대규모 데이터, 피처 상호작용에서 우위")
print("  - FDS에서는 트리 스태킹이 현실적 선택 (실시간 + SHAP)")

---
## 면접 Q&A

### Q: "Self-Attention이 뭔가요?"

> "입력 시퀀스의 각 요소가 다른 모든 요소와의 관계를 학습하는 메커니즘입니다.
> Query, Key, Value를 통해 '어디에 집중할지'를 학습합니다.
> 정형 데이터에서는 각 피처가 다른 피처와의 상호작용을 학습합니다."

### Q: "TabTransformer와 FT-Transformer의 차이는?"

> "TabTransformer는 **범주형 피처만** Transformer로 처리하고, 수치형은 MLP에서 처리합니다.
> FT-Transformer는 **모든 피처**를 Transformer로 처리합니다.
> FT-Transformer가 성능이 더 좋지만, 파라미터가 더 많습니다."

### Q: "정형 데이터에서 Transformer가 트리보다 항상 좋나요?"

> "아닙니다. 2024-2025 벤치마크에서 **작은/중간 데이터에서는 트리가 여전히 우위**입니다.
> Transformer는 100K+ 대규모 데이터, 복잡한 피처 상호작용이 있을 때 유리합니다.
> 또한 트리는 추론 속도가 빠르고 SHAP으로 해석 가능해서 실시간 FDS에 적합합니다."

### Q: "FDS에서 Transformer를 왜 안 썼나요?"

> "세 가지 이유입니다:
> 1. **추론 속도**: 트리(5ms) vs Transformer(50ms+) - 실시간 필요
> 2. **해석 가능성**: 금융 규제로 SHAP 설명 필요 - 트리가 적합
> 3. **성능**: 59만 건에서 트리 스태킹(F1 0.99)이 충분히 높음
>
> 다만 Transformer 실험(1-10)으로 비교는 해볼 계획입니다."

### Q: "Positional Encoding이 정형 데이터에서 왜 불필요한가요?"

> "NLP에서 Positional Encoding은 **단어의 순서** 정보를 제공합니다.
> 정형 데이터에서 피처는 **순서가 없습니다** - [금액, 시간] ≡ [시간, 금액]
> 따라서 Positional Encoding을 추가하면 오히려 불필요한 바이어스가 생깁니다."

---
## 최종 체크포인트

In [ ]:
print("=" * 60)
print("  1-S10 완료: Transformer for Tabular Data")
print("=" * 60)
print()
print("배운 것:")
print()
print("1. Self-Attention")
print("   - Query, Key, Value로 '어디에 집중할지' 학습")
print("   - Multi-Head: 여러 관점에서 관계 학습")
print()
print("2. 정형 데이터에서 Transformer")
print("   - Column Embedding: 피처를 토큰처럼 취급")
print("   - Positional Encoding 불필요 (순서 없음)")
print()
print("3. TabTransformer")
print("   - 범주형 피처만 Transformer 적용")
print("   - 수치형은 MLP에서 처리")
print()
print("4. FT-Transformer")
print("   - 모든 피처를 Transformer로 처리")
print("   - [CLS] 토큰으로 예측")
print()
print("5. 트리 vs Transformer")
print("   - 작은/중간 데이터: 트리 우위")
print("   - 대규모 데이터: Transformer 유리")
print("   - FDS: 트리 스태킹 선택 (속도 + 해석)")
print()
print("=" * 60)
print("다음: 1-10 Transformer 구현 (선택)")
print("=" * 60)